# Tugas 3: Pipeline Pemrosesan ECG
## Full ECG Processing Pipeline

**Mata Kuliah:** Pengolahan Sinyal Medis -- Universitas Indonesia

**Tenggat:** Akhir Minggu 6

---

| | |
|---|---|
| **Nama Mahasiswa** | *(isi nama Anda di sini)* |
| **NPM** | *(isi NPM Anda di sini)* |
| **Kelompok** | *(isi nomor kelompok Anda di sini)* |

---

### Petunjuk Pengerjaan

1. Lengkapi semua bagian yang ditandai dengan `# TODO`.
2. Jawab semua pertanyaan analisis pada sel Markdown yang telah disediakan.
3. Pastikan semua sel kode dapat dijalankan dari awal sampai akhir tanpa error (`Kernel > Restart & Run All`).
4. Kumpulkan notebook ini (file `.ipynb`) sesuai instruksi yang diberikan.

> **Catatan Khusus:** Tugas ini merupakan transisi menuju pengembangan pipeline sebagai script Python. Setelah menyelesaikan notebook ini, Anda akan mengimplementasikan ulang pipeline yang sama sebagai script Python standalone.

> **Catatan Penting — Data:** Tempatkan file data kelompok Anda (`ecg_groupXX.npz`) di folder yang sama dengan notebook ini. Ganti `XX` pada kode pemuatan data dengan nomor kelompok Anda (misalnya `ecg_group03.npz` untuk Kelompok 3).

### Kriteria Penilaian

| Komponen | Bobot |
|---|---|
| Bagian 1: Memuat dan Visualisasi Data ECG | 10% |
| Bagian 2: Preprocessing ECG | 20% |
| Bagian 3: Deteksi R-Peak (Pan-Tompkins) | 20% |
| Bagian 4: Analisis RR Interval dan HRV | 20% |
| Bagian 5: Identifikasi Kelainan ECG | 15% |
| Pertanyaan Analisis | 15% |

In [ ]:
# Impor library yang dibutuhkan
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal as sig

from medsinyal.viz import plot_signal, plot_ecg
from medsinyal.filters import bandpass_filter, notch_filter, remove_baseline_wander
from medsinyal.ecg import (
    detect_r_peaks,
    compute_rr_intervals,
    compute_heart_rate,
    compute_hrv_features,
)

plt.rcParams['figure.dpi'] = 100
plt.rcParams['axes.grid'] = True

---
## Bagian 1: Memuat dan Visualisasi Data ECG (10%)
### Loading and Visualizing ECG Data

Pada bagian ini Anda akan memuat data ECG kelompok Anda dan memahami struktur datanya sebelum membangun pipeline pemrosesan secara lengkap.

Setiap kelompok menerima **dua sinyal ECG mentah** (*raw*):
- `ecg_normal` — sinyal ECG normal (mengandung noise), digunakan sebagai referensi
- `ecg_abnormal` — sinyal ECG abnormal (mengandung noise), merupakan sinyal utama yang dianalisis

Kedua sinyal direkam dengan panjang 15 000 sampel dan *sampling rate* 500 Hz (durasi 30 detik). Tidak ada sinyal bersih (*ground truth*) yang tersedia — Anda akan membangun pipeline pemrosesan dari awal.

**Gambaran Pipeline ECG:**

1. **Pemuatan & Visualisasi Data** — memahami sinyal mentah (*raw*)
2. **Preprocessing** — menghilangkan noise dan artefak
3. **Deteksi R-Peak** — mengidentifikasi puncak QRS
4. **Analisis HRV** — menghitung variabilitas denyut jantung
5. **Identifikasi Kelainan** — menentukan diagnosis berdasarkan seluruh analisis

**Tugas 1.1:** Muat file data kelompok Anda (`ecg_groupXX.npz`) menggunakan `np.load`. Ekstrak `ecg_normal`, `ecg_abnormal`, `fs`, dan `duration`. Buat vektor waktu `t`. Cetak informasi rekaman dan statistik dasar kedua sinyal.

In [ ]:
# TODO: Ganti 'ecg_group01' dengan nama file kelompok Anda (misal: 'ecg_group03')
# data = np.load('ecg_group01.npz')

# TODO: Ekstrak variabel dari file data
# ecg_normal   = data['ecg_normal'].astype(np.float64)    # Sinyal ECG normal (raw)
# ecg_abnormal = data['ecg_abnormal'].astype(np.float64)  # Sinyal ECG abnormal (raw)
# fs           = float(data['fs'])                        # Sampling rate (Hz)
# duration     = float(data['duration'])                  # Durasi sinyal (detik)
# t            = np.arange(len(ecg_normal)) / fs          # Vektor waktu (detik)

# TODO: Cetak informasi rekaman
# n_samples = len(ecg_normal)
# print(f"Informasi Rekaman:")
# print(f"  Sampling rate  : {fs} Hz")
# print(f"  Jumlah sampel  : {n_samples}")
# print(f"  Durasi         : {duration:.1f} detik")
# print(f"  Resolusi waktu : {1000/fs:.2f} ms/sampel")
# print(f"\nStatistik sinyal normal  : mean={np.mean(ecg_normal):.4f}  std={np.std(ecg_normal):.4f}  range=[{np.min(ecg_normal):.2f}, {np.max(ecg_normal):.2f}]")
# print(f"Statistik sinyal abnormal: mean={np.mean(ecg_abnormal):.4f}  std={np.std(ecg_abnormal):.4f}  range=[{np.min(ecg_abnormal):.2f}, {np.max(ecg_abnormal):.2f}]")

**Tugas 1.2:** Plot 5 detik pertama dari `ecg_normal` dan `ecg_abnormal` secara berdampingan dalam 2 subplot. Amati perbedaan visual antara kedua sinyal sebelum preprocessing. Identifikasi jenis noise yang terlihat (baseline wander, interferensi 50 Hz, random noise). Tambahkan subplot ketiga yang menampilkan kedua sinyal secara *overlaid* (ditumpuk) untuk perbandingan langsung.

In [ ]:
# TODO: Tentukan batas indeks untuk 5 detik pertama
# n_5sec = int(5 * fs)
# t_5sec = t[:n_5sec]
# ecg_normal_5sec   = ecg_normal[:n_5sec]
# ecg_abnormal_5sec = ecg_abnormal[:n_5sec]

# TODO: Plot dalam 3 subplot
# fig, axes = plt.subplots(3, 1, figsize=(14, 9), sharex=True)
#
# axes[0].plot(t_5sec, ecg_normal_5sec, 'steelblue', linewidth=0.8)
# axes[0].set_title('ECG Normal (Raw) — 5 Detik Pertama')
# axes[0].set_ylabel('Amplitudo')
#
# axes[1].plot(t_5sec, ecg_abnormal_5sec, 'tomato', linewidth=0.8)
# axes[1].set_title('ECG Abnormal (Raw) — 5 Detik Pertama')
# axes[1].set_ylabel('Amplitudo')
#
# axes[2].plot(t_5sec, ecg_normal_5sec,   'steelblue', linewidth=0.8, alpha=0.7, label='Normal')
# axes[2].plot(t_5sec, ecg_abnormal_5sec, 'tomato',    linewidth=0.8, alpha=0.7, label='Abnormal')
# axes[2].set_title('Perbandingan Normal vs Abnormal (Overlaid)')
# axes[2].set_ylabel('Amplitudo')
# axes[2].set_xlabel('Waktu (detik)')
# axes[2].legend()
#
# plt.tight_layout()
# plt.show()

# TODO: Jelaskan noise apa saja yang Anda identifikasi pada masing-masing sinyal

---
## Bagian 2: Preprocessing ECG (20%)
### ECG Preprocessing

Preprocessing ECG bertujuan menghilangkan artefak dan noise agar sinyal siap untuk analisis lebih lanjut. Tiga tahap utama preprocessing:

1. **Notch filter 50 Hz** — menghilangkan *power line interference* (gangguan jala-jala listrik PLN)
2. **Highpass / Baseline removal 0.5 Hz** — menghilangkan *baseline wander* akibat gerakan atau pernapasan
3. **Bandpass filter 0.5–40 Hz** — mempertahankan komponen frekuensi yang relevan secara klinis

Sinyal target utama adalah `ecg_abnormal`. Sinyal `ecg_normal` juga diproses (dalam satu baris) untuk digunakan sebagai referensi perbandingan pada bagian berikutnya.

**Tugas 2.1:** Implementasikan preprocessing langkah demi langkah secara manual (tanpa menggunakan wrapper `preprocess_ecg`) pada sinyal `ecg_abnormal`. Terapkan ketiga filter secara berurutan dan plot hasilnya dalam 4 subplot (sinyal asli + 3 tahap).

In [ ]:
# TODO: Step 1 -- Terapkan notch filter untuk menghilangkan power line 50 Hz
# ecg_notched = notch_filter(ecg_abnormal, 50, fs)

# TODO: Step 2 -- Hilangkan baseline wander
# ecg_no_baseline = remove_baseline_wander(ecg_notched, fs)

# TODO: Step 3 -- Terapkan bandpass filter 0.5-40 Hz
# ecg_abnormal_preprocessed = bandpass_filter(ecg_no_baseline, 0.5, 40, fs)

# TODO: Proses sinyal normal dalam satu baris (untuk referensi perbandingan)
# ecg_normal_preprocessed = bandpass_filter(remove_baseline_wander(notch_filter(ecg_normal, 50, fs), fs), 0.5, 40, fs)

# TODO: Plot keempat sinyal (asli + 3 tahap) dalam 5 detik pertama
# n_5sec = int(5 * fs)
# fig, axes = plt.subplots(4, 1, figsize=(14, 12), sharex=True)
# labels  = ['Raw (ecg_abnormal)', 'Setelah Notch 50 Hz', 'Setelah Baseline Removal', 'Setelah Bandpass 0.5–40 Hz']
# signals = [ecg_abnormal, ecg_notched, ecg_no_baseline, ecg_abnormal_preprocessed]
# colors  = ['gray', 'orange', 'steelblue', 'green']
# for ax, label, s, c in zip(axes, labels, signals, colors):
#     ax.plot(t[:n_5sec], s[:n_5sec], color=c, linewidth=0.8)
#     ax.set_title(label)
#     ax.set_ylabel('Amplitudo')
# axes[-1].set_xlabel('Waktu (detik)')
# plt.tight_layout()
# plt.show()

**Tugas 2.2:** Evaluasi pengaruh setiap tahap preprocessing secara kuantitatif. Karena tidak ada sinyal bersih sebagai *ground truth*, gunakan pendekatan **energi noise yang dihilangkan**: hitung selisih antara sinyal sebelum dan sesudah setiap tahap, lalu hitung energinya.

$$E_{\text{removed}} = \frac{1}{N}\sum_{n=1}^{N}\left(x_{\text{before}}[n] - x_{\text{after}}[n]\right)^2$$

Tampilkan hasilnya dalam bentuk tabel untuk mengetahui tahap mana yang paling banyak menghilangkan noise.

In [ ]:
# TODO: Hitung energi noise yang dihilangkan di setiap tahap
# def energy_removed(before, after):
#     noise = before - after
#     return np.mean(noise ** 2)
#
# e1 = energy_removed(ecg_abnormal,      ecg_notched)           # Notch filter
# e2 = energy_removed(ecg_notched,       ecg_no_baseline)       # Baseline removal
# e3 = energy_removed(ecg_no_baseline,   ecg_abnormal_preprocessed)  # Bandpass
#
# print("Energi Noise yang Dihilangkan di Setiap Tahap:")
# print("-" * 55)
# print(f"{'Tahap':<35} {'Energi Removed':>18}")
# print("-" * 55)
# print(f"{'Notch filter 50 Hz':<35} {e1:>18.6f}")
# print(f"{'Baseline wander removal':<35} {e2:>18.6f}")
# print(f"{'Bandpass filter 0.5-40 Hz':<35} {e3:>18.6f}")
# print("-" * 55)
# print(f"{'Total':<35} {e1+e2+e3:>18.6f}")

**Tugas 2.3:** Terapkan pipeline preprocessing yang sama pada `ecg_normal` dan tampilkan kedua sinyal hasil preprocessing (`ecg_normal_preprocessed` dan `ecg_abnormal_preprocessed`) secara *overlaid* dalam satu plot. Amati perbedaan antara sinyal normal dan abnormal setelah noise dihilangkan.

In [ ]:
# (ecg_normal_preprocessed sudah dihitung pada Tugas 2.1)

# TODO: Plot kedua sinyal preprocessed secara overlaid (5 detik pertama)
# n_5sec = int(5 * fs)
# fig, ax = plt.subplots(figsize=(14, 4))
# ax.plot(t[:n_5sec], ecg_normal_preprocessed[:n_5sec],   'steelblue', linewidth=0.9, alpha=0.8, label='Normal (preprocessed)')
# ax.plot(t[:n_5sec], ecg_abnormal_preprocessed[:n_5sec], 'tomato',    linewidth=0.9, alpha=0.8, label='Abnormal (preprocessed)')
# ax.set_title('Perbandingan ECG Normal vs Abnormal Setelah Preprocessing')
# ax.set_xlabel('Waktu (detik)')
# ax.set_ylabel('Amplitudo')
# ax.legend()
# plt.tight_layout()
# plt.show()

# TODO: Deskripsikan perbedaan yang Anda amati antara kedua sinyal

---
## Bagian 3: Deteksi R-Peak (Pan-Tompkins) (20%)
### R-Peak Detection

Algoritma **Pan-Tompkins** (1985) adalah metode klasik dan masih banyak digunakan untuk deteksi kompleks QRS pada sinyal ECG. Algoritma ini bekerja dalam lima langkah:

1. **Bandpass filter 5–15 Hz** — memperkuat energi QRS dan melemahkan gelombang P, T, dan noise
2. **Diferensiasi** — memperkuat kemiringan (*slope*) tajam dari kompleks QRS
3. **Squaring** — memastikan semua nilai positif dan memperkuat puncak besar secara non-linear
4. **Moving window integration** — menghaluskan sinyal dan menciptakan puncak yang lebih lebar
5. **Thresholding adaptif** — menentukan puncak R berdasarkan ambang batas yang diperbarui secara dinamis

**Tugas 3.1:** Implementasikan versi sederhana algoritma Pan-Tompkins secara manual pada `ecg_abnormal_preprocessed`. Plot sinyal antara (*intermediate signals*) dari setiap langkah untuk 3 detik pertama dalam 4 subplot.

In [ ]:
# TODO: Implementasikan Pan-Tompkins secara manual

# Step 1: Bandpass filter 5-15 Hz untuk memperkuat energi QRS
# ecg_bp = bandpass_filter(ecg_abnormal_preprocessed, 5.0, 15.0, fs, order=2)

# Step 2: Diferensiasi -- menghitung turunan untuk memperkuat kemiringan QRS
# ecg_diff = np.diff(ecg_bp)
# ecg_diff = np.append(ecg_diff, ecg_diff[-1])  # Pertahankan panjang sinyal

# Step 3: Squaring -- memastikan semua nilai positif dan memperkuat puncak besar
# ecg_squared = ecg_diff ** 2

# Step 4: Moving window integration -- menghaluskan sinyal
# window_size = int(0.15 * fs)  # Window 150 ms
# ecg_integrated = np.convolve(ecg_squared, np.ones(window_size) / window_size, mode='same')

# TODO: Plot 4 sinyal antara dalam 3 detik pertama
# n_3sec = int(3 * fs)
# fig, axes = plt.subplots(4, 1, figsize=(14, 10), sharex=True)
# labels  = ['Preprocessed', 'Setelah Bandpass 5-15 Hz', 'Setelah Squaring', 'Setelah Moving Avg Integration']
# signals = [ecg_abnormal_preprocessed, ecg_bp, ecg_squared, ecg_integrated]
# for ax, label, s in zip(axes, labels, signals):
#     ax.plot(t[:n_3sec], s[:n_3sec], linewidth=0.8)
#     ax.set_title(label)
#     ax.set_ylabel('Amplitudo')
# axes[-1].set_xlabel('Waktu (detik)')
# plt.tight_layout()
# plt.show()

**Tugas 3.2:** Gunakan fungsi `detect_r_peaks` dari `medsinyal` untuk mendeteksi R-peak pada kedua sinyal preprocessed (`ecg_abnormal_preprocessed` dan `ecg_normal_preprocessed`). Plot hasilnya menggunakan `plot_ecg` dengan marker R-peak.

In [ ]:
# TODO: Deteksi R-peak pada sinyal abnormal
# r_peaks_abnormal = detect_r_peaks(ecg_abnormal_preprocessed, fs)
# print(f"Jumlah R-peak terdeteksi (abnormal): {len(r_peaks_abnormal)}")

# TODO: Deteksi R-peak pada sinyal normal (untuk perbandingan)
# r_peaks_normal = detect_r_peaks(ecg_normal_preprocessed, fs)
# print(f"Jumlah R-peak terdeteksi (normal)  : {len(r_peaks_normal)}")

# TODO: Plot ECG abnormal preprocessed dengan R-peak
# plot_ecg(
#     t,
#     ecg_abnormal_preprocessed,
#     r_peaks=r_peaks_abnormal,
#     title='ECG Abnormal Preprocessed dengan R-Peak Terdeteksi',
#     fs=fs,
# )

# TODO: Plot ECG normal preprocessed dengan R-peak
# plot_ecg(
#     t,
#     ecg_normal_preprocessed,
#     r_peaks=r_peaks_normal,
#     title='ECG Normal Preprocessed dengan R-Peak Terdeteksi',
#     fs=fs,
# )

**Tugas 3.3:** Evaluasi kelayakan (*plausibility*) hasil deteksi R-peak. Karena tidak ada *ground truth*, gunakan kriteria fisiologis: hitung *heart rate* perkiraan dari RR interval dan periksa apakah nilainya masuk rentang normal (40–200 BPM). Plot R-peak yang terdeteksi pada kedua sinyal secara berdampingan.

In [ ]:
# TODO: Hitung heart rate perkiraan dari R-peak yang terdeteksi
# if len(r_peaks_abnormal) > 1:
#     rr_approx = np.diff(r_peaks_abnormal) / fs  # RR intervals in seconds
#     hr_approx = 60.0 / np.mean(rr_approx)
#     print(f"Heart rate perkiraan (abnormal): {hr_approx:.1f} BPM")
#     print(f"Jumlah R-peak terdeteksi       : {len(r_peaks_abnormal)}")
#     print(f"Jumlah R-peak expected (30s)   : ~{hr_approx/60*30:.0f} beats")
#
# # TODO: Bandingkan dengan sinyal normal
# if len(r_peaks_normal) > 1:
#     rr_normal = np.diff(r_peaks_normal) / fs
#     hr_normal = 60.0 / np.mean(rr_normal)
#     print(f"\nHeart rate perkiraan (normal)  : {hr_normal:.1f} BPM")
#     print(f"Jumlah R-peak terdeteksi       : {len(r_peaks_normal)}")

# TODO: Plot R-peak pada kedua sinyal berdampingan (5 detik pertama)
# n_5sec = int(5 * fs)
# fig, axes = plt.subplots(2, 1, figsize=(14, 7), sharex=True)
#
# # Filter R-peaks dalam 5 detik pertama
# rp_norm_5s = r_peaks_normal[r_peaks_normal < n_5sec]
# rp_abn_5s  = r_peaks_abnormal[r_peaks_abnormal < n_5sec]
#
# axes[0].plot(t[:n_5sec], ecg_normal_preprocessed[:n_5sec], 'steelblue', linewidth=0.8)
# axes[0].scatter(t[rp_norm_5s], ecg_normal_preprocessed[rp_norm_5s], color='red', zorder=5, label='R-peak')
# axes[0].set_title('ECG Normal — R-Peak Terdeteksi')
# axes[0].set_ylabel('Amplitudo')
# axes[0].legend()
#
# axes[1].plot(t[:n_5sec], ecg_abnormal_preprocessed[:n_5sec], 'tomato', linewidth=0.8)
# axes[1].scatter(t[rp_abn_5s], ecg_abnormal_preprocessed[rp_abn_5s], color='darkred', zorder=5, label='R-peak')
# axes[1].set_title('ECG Abnormal — R-Peak Terdeteksi')
# axes[1].set_ylabel('Amplitudo')
# axes[1].set_xlabel('Waktu (detik)')
# axes[1].legend()
#
# plt.tight_layout()
# plt.show()

---
## Bagian 4: Analisis RR Interval dan HRV (20%)
### RR Interval and HRV Analysis

**RR interval** adalah jarak waktu antara dua R-peak yang berurutan, dan merupakan dasar dari analisis *Heart Rate Variability* (HRV). Nilai RR interval normal saat istirahat berkisar antara **0.6–1.0 detik** (60–100 BPM).

HRV mencerminkan aktivitas sistem saraf otonom (ANS) yang mengatur jantung. Beberapa fitur HRV domain waktu yang umum digunakan:

| Fitur | Deskripsi | Rentang Normal |
|---|---|---|
| `mean_rr` | Rata-rata RR interval | 600–1000 ms |
| `sdnn` | Standar deviasi semua RR interval | 50–100 ms |
| `rmssd` | Root mean square of successive differences | 20–80 ms |
| `pnn50` | Persentase perbedaan RR > 50 ms | 5–50 % |
| `mean_hr` | Rata-rata heart rate | 60–100 BPM |

**Tugas 4.1:** Hitung RR interval menggunakan `compute_rr_intervals(r_peaks_abnormal, fs)` dan *instantaneous heart rate* menggunakan `compute_heart_rate(rr_intervals)`. Plot takogram RR interval dan *instantaneous heart rate* dengan garis referensi nilai normal.

In [ ]:
# TODO: Hitung RR intervals dari sinyal abnormal
# rr_intervals = compute_rr_intervals(r_peaks_abnormal, fs)
# print(f"Jumlah RR interval: {len(rr_intervals)}")
# print(f"RR interval rata-rata: {np.mean(rr_intervals)*1000:.1f} ms")
# print(f"RR interval min      : {np.min(rr_intervals)*1000:.1f} ms")
# print(f"RR interval max      : {np.max(rr_intervals)*1000:.1f} ms")

# TODO: Hitung instantaneous heart rate
# hr_instantaneous = compute_heart_rate(rr_intervals)

# TODO: Plot takogram RR interval
# beat_times = t[r_peaks_abnormal[1:]]  # Waktu beat (dimulai dari beat ke-2)
# fig, axes = plt.subplots(2, 1, figsize=(12, 6), sharex=True)
#
# axes[0].plot(beat_times, rr_intervals * 1000, 'o-', color='steelblue', linewidth=1, markersize=3)
# axes[0].axhline(600,  color='green', linestyle='--', alpha=0.6, label='Normal min (600 ms)')
# axes[0].axhline(1000, color='green', linestyle='--', alpha=0.6, label='Normal max (1000 ms)')
# axes[0].set_title('Tachogram RR Interval — ECG Abnormal')
# axes[0].set_ylabel('RR Interval (ms)')
# axes[0].legend()
#
# axes[1].plot(beat_times, hr_instantaneous, 'o-', color='tomato', linewidth=1, markersize=3)
# axes[1].axhline(60,  color='green', linestyle='--', alpha=0.6, label='Normal min (60 BPM)')
# axes[1].axhline(100, color='green', linestyle='--', alpha=0.6, label='Normal max (100 BPM)')
# axes[1].set_title('Instantaneous Heart Rate — ECG Abnormal')
# axes[1].set_ylabel('Heart Rate (BPM)')
# axes[1].set_xlabel('Waktu (detik)')
# axes[1].legend()
#
# plt.tight_layout()
# plt.show()

**Tugas 4.2:** Hitung fitur-fitur HRV menggunakan `compute_hrv_features(rr_intervals)`. Tampilkan semua fitur dalam tabel berformat rapi dan interpretasikan nilainya terhadap rentang normal.

In [ ]:
# TODO: Hitung fitur HRV untuk sinyal abnormal
# hrv_features = compute_hrv_features(rr_intervals)

# TODO: Tampilkan semua fitur dalam tabel berformat
# print("Fitur HRV — ECG Abnormal:")
# print("-" * 60)
# print(f"{'Fitur':<20} {'Nilai':>10} {'Satuan':>8} {'Normal':>18}")
# print("-" * 60)
# print(f"{'mean_rr':<20} {hrv_features.get('mean_rr',0)*1000:>10.1f} {'ms':>8} {'600-1000 ms':>18}")
# print(f"{'sdnn':<20} {hrv_features.get('sdnn',0)*1000:>10.1f} {'ms':>8} {'50-100 ms':>18}")
# print(f"{'rmssd':<20} {hrv_features.get('rmssd',0)*1000:>10.1f} {'ms':>8} {'20-80 ms':>18}")
# print(f"{'pnn50':<20} {hrv_features.get('pnn50',0):>10.1f} {'%':>8} {'5-50 %':>18}")
# print(f"{'mean_hr':<20} {hrv_features.get('mean_hr',0):>10.1f} {'BPM':>8} {'60-100 BPM':>18}")
# print(f"{'std_hr':<20} {hrv_features.get('std_hr',0):>10.1f} {'BPM':>8} {'-':>18}")

**Tugas 4.3:** Bandingkan fitur HRV antara sinyal normal dan sinyal abnormal kelompok Anda. Tampilkan perbandingan dalam bentuk tabel dan interpretasikan perbedaannya.

In [ ]:
# TODO: Hitung HRV untuk sinyal normal (sebagai referensi)
# rr_normal_intervals = compute_rr_intervals(r_peaks_normal, fs)
# hrv_normal = compute_hrv_features(rr_normal_intervals)

# TODO: Tampilkan tabel perbandingan HRV normal vs abnormal
# print(f"{'Fitur':<20} {'Normal':>15} {'Abnormal':>15} {'Satuan':>8}")
# print("-" * 62)
# features_to_compare = ['mean_rr', 'sdnn', 'rmssd', 'pnn50', 'mean_hr', 'std_hr']
# units  = ['ms', 'ms', 'ms', '%', 'BPM', 'BPM']
# scales = [1000, 1000, 1000, 1, 1, 1]  # Convert rr from seconds to ms
# for feat, unit, scale in zip(features_to_compare, units, scales):
#     n_val = hrv_normal.get(feat, 0) * scale
#     a_val = hrv_features.get(feat, 0) * scale
#     print(f"{feat:<20} {n_val:>15.2f} {a_val:>15.2f} {unit:>8}")

---
## Bagian 5: Identifikasi Kelainan ECG (15%)
### ECG Abnormality Identification

Berdasarkan seluruh analisis yang telah Anda lakukan — visualisasi sinyal, preprocessing, deteksi R-peak, tachogram RR interval, dan perbandingan fitur HRV — jawab pertanyaan berikut tentang sinyal abnormal kelompok Anda.

**Tugas 5.1:** Rangkum temuan utama Anda dari analisis sinyal abnormal. Isi setiap variabel di bawah ini berdasarkan hasil analisis Anda.

In [ ]:
# TODO: Rangkum temuan utama Anda dari sinyal abnormal
# Isi setiap variabel di bawah ini berdasarkan hasil analisis Anda

# Apakah irama jantung reguler atau tidak teratur?
# irama = 'reguler'  # atau 'tidak teratur'

# Berapa heart rate rata-rata pada sinyal abnormal?
# hr_rata_rata = ...  # BPM

# Apakah morfologi QRS terlihat normal atau berbeda?
# qrs_morfologi = 'normal'  # atau 'lebar', 'berbeda', dll.

# Apakah ada perubahan segmen ST yang terlihat?
# segmen_st = 'normal'  # atau 'elevasi', 'depresi'

# Apakah ada gelombang P yang terlihat jelas sebelum setiap QRS?
# gelombang_p = 'terlihat'  # atau 'tidak terlihat', 'tidak konsisten'

# print("Ringkasan Temuan:")
# print(f"  Irama          : {irama}")
# print(f"  Heart rate     : {hr_rata_rata} BPM")
# print(f"  Morfologi QRS  : {qrs_morfologi}")
# print(f"  Segmen ST      : {segmen_st}")
# print(f"  Gelombang P    : {gelombang_p}")

**Tugas 5.2:** Berdasarkan temuan di atas, diagnosis apa yang paling mungkin untuk sinyal abnormal kelompok Anda? Jelaskan alasannya mengacu pada temuan kuantitatif (heart rate, fitur HRV, RR interval) dan kualitatif (morfologi QRS, gelombang P, segmen ST).

**Jawaban:** *(Tulis diagnosis dan penjelasan Anda di sini)*

---
## Pertanyaan Analisis (15%)
### Analysis Questions

Jawab pertanyaan-pertanyaan berikut berdasarkan pemahaman Anda dari hasil percobaan dan materi kuliah. Jawab dengan lengkap dan jelas.

### Pertanyaan 1

Dalam preprocessing ECG, urutan penerapan filter sangat penting. Mengapa lebih baik menerapkan *notch filter* (50 Hz) **sebelum** *highpass filter* (*baseline removal*), bukan sebaliknya? Jelaskan dengan mempertimbangkan karakteristik frekuensi masing-masing filter dan bagaimana urutan yang berbeda dapat mempengaruhi hasil preprocessing.

**Jawaban:**

*(Tulis jawaban Anda di sini)*

### Pertanyaan 2

Algoritma Pan-Tompkins menggunakan kombinasi *bandpass filter*, *differentiator*, dan *squaring* sebelum integrasi. Jelaskan peran masing-masing langkah:

**(a)** Mengapa *bandpass filter* 5–15 Hz dipilih untuk memperkuat kompleks QRS? Apa yang terjadi pada gelombang P dan T setelah filter ini?

**(b)** Apa peran *differentiator* dalam mendeteksi puncak R? Mengapa kemiringan (*slope*) QRS menjadi fitur penting?

**(c)** Mengapa hasil diferensiasi di-*squaring* sebelum integrasi? Apa efeknya terhadap puncak positif dan negatif?

**Jawaban:**

*(a)* *(Tulis jawaban Anda di sini)*

*(b)* *(Tulis jawaban Anda di sini)*

*(c)* *(Tulis jawaban Anda di sini)*

### Pertanyaan 3

Berdasarkan hasil analisis Anda di Bagian 1–5:

**(a)** Berapa heart rate rata-rata sinyal normal kelompok Anda? Sinyal abnormal? Apakah ada perbedaan signifikan?

**(b)** Bandingkan nilai SDNN dan RMSSD antara sinyal normal dan abnormal. Apa yang dapat Anda simpulkan tentang variabilitas denyut jantung pada kedua sinyal?

**(c)** Berdasarkan semua analisis Anda (Bagian 1–5), diagnosis apa yang Anda ajukan untuk sinyal abnormal kelompok Anda? Sebutkan minimal 3 temuan yang mendukung diagnosis tersebut.

**Jawaban:**

*(a)* *(Tulis jawaban Anda di sini)*

*(b)* *(Tulis jawaban Anda di sini)*

*(c)* *(Tulis jawaban Anda di sini)*

---
## Bonus: Transisi ke Script Python
### Transition to Python Script

Setelah menyelesaikan notebook ini, implementasikan ulang pipeline ECG Anda sebagai script Python standalone. Script ini merupakan langkah awal transisi dari *exploratory notebook* menuju kode produksi yang terstruktur.

**Spesifikasi script `ecg_pipeline.py`:**
- **Input**: path ke file `.npz` ECG (via argumen `--input`)
- **Output**: print ringkasan ke terminal (heart rate, fitur HRV, jumlah R-peak terdeteksi)
- **Simpan** di: `assignments/tugas03_ecg_pipeline/ecg_pipeline.py`

**Contoh penggunaan:**
```bash
python ecg_pipeline.py --input ecg_group01.npz
```

**Contoh output yang diharapkan:**
```
=== ECG Pipeline Summary ===
File         : ecg_group01.npz
Duration     : 30.00 s
R-peaks      : 42
Mean HR      : 72.5 BPM
SDNN         : 65.3 ms
RMSSD        : 38.1 ms
pNN50        : 18.4 %
===========================
```

Ini adalah latihan untuk transisi dari notebook ke script produksi yang akan lebih sering digunakan di paruh kedua kuliah, terutama saat membangun pipeline pemrosesan sinyal yang dapat dijalankan secara otomatis (*batch processing*).

In [ ]:
# TODO (Bonus): Buat kerangka script ecg_pipeline.py
# Salin kode di bawah ini ke file ecg_pipeline.py dan lengkapi bagian yang kosong

script_template = '''
#!/usr/bin/env python
"""ECG Processing Pipeline Script.

Usage:
    python ecg_pipeline.py --input ecg_group01.npz
"""
import argparse
import numpy as np
from medsinyal.filters import bandpass_filter, notch_filter, remove_baseline_wander
from medsinyal.ecg import detect_r_peaks, compute_rr_intervals
from medsinyal.ecg import compute_heart_rate, compute_hrv_features


def run_pipeline(input_path: str) -> None:
    """Jalankan pipeline ECG dan cetak ringkasan."""
    # TODO: Muat data
    # data = np.load(input_path)

    # TODO: Ekstrak variabel
    # ecg_abnormal = data['ecg_abnormal'].astype(np.float64)
    # fs       = float(data['fs'])
    # duration = float(data['duration'])
    # t        = np.arange(len(ecg_abnormal)) / fs

    # TODO: Preprocessing
    # ecg_notched     = notch_filter(ecg_abnormal, 50, fs)
    # ecg_no_baseline = remove_baseline_wander(ecg_notched, fs)
    # ecg_preprocessed = bandpass_filter(ecg_no_baseline, 0.5, 40, fs)

    # TODO: Deteksi R-peak
    # r_peaks = detect_r_peaks(ecg_preprocessed, fs)

    # TODO: Hitung RR interval dan HRV
    # rr  = compute_rr_intervals(r_peaks, fs)
    # hrv = compute_hrv_features(rr)

    # TODO: Cetak ringkasan
    # print("=== ECG Pipeline Summary ===")
    # print(f"File         : {input_path}")
    # print(f"Duration     : {duration:.2f} s")
    # print(f"R-peaks      : {len(r_peaks)}")
    # print(f"Mean HR      : {hrv.get('mean_hr', 0):.1f} BPM")
    # print(f"SDNN         : {hrv.get('sdnn', 0)*1000:.1f} ms")
    # print(f"RMSSD        : {hrv.get('rmssd', 0)*1000:.1f} ms")
    # print(f"pNN50        : {hrv.get('pnn50', 0):.1f} %")
    # print("===========================")
    pass


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="ECG Processing Pipeline")
    parser.add_argument("--input", required=True, help="Path ke file .npz ECG kelompok")
    args = parser.parse_args()
    run_pipeline(args.input)
'''

print("Kerangka script ecg_pipeline.py:")
print(script_template)

---

## Checklist Sebelum Pengumpulan

Pastikan Anda telah:

- [ ] Mengisi nama, NPM, dan nomor kelompok di bagian header
- [ ] Menempatkan file `ecg_groupXX.npz` di folder yang sama dengan notebook
- [ ] Menyelesaikan Bagian 1: memuat data kelompok dan plot 5 detik pertama (normal + abnormal)
- [ ] Menyelesaikan Bagian 2: preprocessing manual (3 langkah) dan evaluasi energi noise
- [ ] Menyelesaikan Bagian 3: implementasi Pan-Tompkins manual dan evaluasi kelayakan deteksi
- [ ] Menyelesaikan Bagian 4: analisis RR interval, fitur HRV, dan perbandingan normal vs abnormal
- [ ] Menyelesaikan Bagian 5: identifikasi kelainan dan diagnosis sinyal abnormal kelompok Anda
- [ ] Menjawab semua pertanyaan analisis (Pertanyaan 1, 2, dan 3)
- [ ] Menjalankan seluruh notebook dari awal (`Kernel > Restart & Run All`) tanpa error
- [ ] Semua plot tampil dengan benar dan memiliki label sumbu serta judul
- [ ] (Bonus) Membuat `ecg_pipeline.py` yang dapat dijalankan dari terminal

---
*Tugas 3 -- Pengolahan Sinyal Medis -- Universitas Indonesia*